In [ ]:
#Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install opencv-python
!pip install tensorflow_hub

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [ ]:
# Charger le modèle de détection SSD MobileNet V2
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")

In [ ]:
# Liste des noms de classes (80 classes COCO)
classe = np.array([
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
])
classes = np.concatenate([classe, np.repeat('none', 100)])


In [ ]:
# 🔁 À adapter : ton dossier avec des images dans Google Drive
folder_path = "/content/drive/MyDrive/Colab_Notebooks/images"  # ← change ça

In [ ]:
def process_local_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erreur de lecture : {image_path}")
        return
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_rgb)[tf.newaxis, ...]

    detections = detector(input_tensor)
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    classes_detected = detections['detection_classes'][0].numpy().astype(np.uint8)

    height, width, _ = image.shape
    for i in range(len(scores)):
        if scores[i] < 0.3:
            continue
        box = boxes[i] * np.array([height, width, height, width])
        box = box.astype(np.int32)
        cv2.rectangle(image_rgb, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
        label = f"{classes[classes_detected[i]]}: {scores[i]*100:.1f}%"
        cv2.putText(image_rgb, label, (box[1], box[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"Détection : {os.path.basename(image_path)}")
    plt.show()


In [ ]:
supported_formats = ('.jpg', '.jpeg', '.png', '.bmp')
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(supported_formats)]

print(f"{len(image_files)} images trouvées dans le dossier.")

for image_path in image_files:
    process_local_image(image_path)
